In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')


import re
from konlpy.tag import Okt, Mecab
from ckonlpy.tag import Twitter
from hanspell import spell_checker
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x).lower())
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,various artists
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,murray perahia
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,peter gabriel
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,matoma
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,jude law


In [2]:
train = pd.read_json('data/train.json')
train['tags__'] = train['tags'].apply(lambda x : " ".join(x))
train['plylst_title__'] = train['plylst_title']
train

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락,여행같은 음악
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상,요즘 너 말야
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한,"편하게, 잔잔하게 들을 수 있는 곡.-"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,크리스마스 분위기에 흠뻑 취하고 싶을때
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스,추억의 노래 ㅋ
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,록메탈 밴드사운드 록 락메탈 메탈 락 extreme,METAL E'SM #2
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,일렉,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,담시 가족 눈물 그리움 주인공 나의_이야기 사랑 친구,#1. 눈물이 앞을 가리는 나의_이야기
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,잔잔한 버스 퇴근버스 Pop 풍경 퇴근길,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP


In [3]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

okt = Okt()

twitter = Twitter()
# 0살부터 99살까지 명사로 추가해줌
twitter.add_dictionary([str(x) + '살' for x in range(1,100)], 'Noun')
twitter.add_dictionary(['10대','20대','30대','40대','50대','60대','70대'], 'Noun')
# 특정 가수 입력
twitter.add_dictionary('아이오아이', 'Noun')

m = Mecab(r'C:/mecab/mecab-ko-dic')

In [5]:
song_meta.query('artist_name == "백현 (baekhyun)"')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
3934,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Psycho (Bonus Track),"[GN2500, GN0200]",[백현 (BAEKHYUN)],3934,백현 (baekhyun)
51681,"[GN2502, GN2501, GN0303, GN2504, GN0301]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Stay Up (Feat. Beenzino),"[GN2500, GN0300]",[백현 (BAEKHYUN)],51681,백현 (baekhyun)
125181,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Diamond,"[GN0400, GN2500]",[백현 (BAEKHYUN)],125181,백현 (baekhyun)
186196,"[GN2502, GN2501, GN0101, GN2505]",20170414,바래다줄게 (Take You Home) - SM STATION,10055218,[672859],바래다줄게 (Take You Home) (Inst.),"[GN2500, GN0100]",[백현 (BAEKHYUN)],186196,백현 (baekhyun)
312216,"[GN0105, GN1501, GN0101, GN1504]",20200225,낭만닥터 김사부 2 OST,10393897,[672859],너를 사랑하고 있어,"[GN1500, GN0100]",[백현 (BAEKHYUN)],312216,백현 (baekhyun)
486065,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],Ice Queen,"[GN0400, GN2500]",[백현 (BAEKHYUN)],486065,백현 (baekhyun)
505587,"[GN0401, GN2502, GN2501, GN0402]",20190710,City Lights - The 1st Mini Album,10305581,[672859],UN Village,"[GN0400, GN2500]",[백현 (BAEKHYUN)],505587,백현 (baekhyun)
532199,"[GN0105, GN0101, GN1504, GN2502, GN1501, GN250...",20200107,낭만닥터 김사부 2 OST Part.1,10372655,[672859],너를 사랑하고 있어,"[GN2500, GN1500, GN0100]",[백현 (BAEKHYUN)],532199,백현 (baekhyun)
572181,"[GN0105, GN0101]",20151228,2015 가요대전 Limited Edition,2658903,[672859],비처럼 음악처럼,[GN0100],[백현 (BAEKHYUN)],572181,백현 (baekhyun)
607659,"[GN0105, GN0101, GN2505, GN2502, GN1501, GN250...",20200229,하이에나 OST Part.2,10396078,[672859],너에게 가는 이 길 위에서 (너.이.길),"[GN2500, GN1500, GN0100]",[백현 (BAEKHYUN)],607659,백현 (baekhyun)


In [6]:
# 참고--
# 포함 문자열 찾기
temp_str = 'December Holiday Season is finally here!'

for temp in ['Decem', 'mb','day', 'Sea']:
    if temp in temp_str:
        print(temp_str)

December Holiday Season is finally here!
December Holiday Season is finally here!
December Holiday Season is finally here!
December Holiday Season is finally here!


### 해당 가수 찾아오기

In [7]:
from itertools import chain

In [8]:
# 해당 가수 찾아오기
from itertools import chain

def find_singer(names = []):
    
    names = names.split()
    
    idx = []
    for name in names:
        A = [x for x in song_meta['artist_name'] if name in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(song_meta[song_meta['artist_name'] == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return song_meta.iloc[idx_list,:]

In [9]:
find_singer('백현')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name
157484,"[GN1502, GN1501]",20021210,철없는 아내와 파란만장한 남편 그리고 태권소녀 OST,25904,"[2268, 168436]",구멍난 그림자,[GN1500],"[한대수, 백현진]",157484,한대수 백현진
180023,"[GN0105, GN0101]",20170214,비가와,10038305,"[490981, 672859]",비가와 (Rain),[GN0100],"[소유 (SOYOU), 백현 (BAEKHYUN)]",180023,소유 (soyou) 백현 (baekhyun)
121303,"[GN0105, GN0101]",20160513,The Day - SM STATION,2684689,"[175139, 672859]",The Day,[GN0100],"[케이윌, 백현 (BAEKHYUN)]",121303,케이윌 백현 (baekhyun)
468075,[GN0101],20160513,The Day - SM STATION,2684689,"[175139, 672859]",The Day (Inst.),[GN0100],"[케이윌, 백현 (BAEKHYUN)]",468075,케이윌 백현 (baekhyun)
239509,[GN0701],20070312,The Unique,347021,[221002],그리운 그사람,[GN0700],[백현우],239509,백현우
...,...,...,...,...,...,...,...,...,...,...
556642,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20110218,찰라의 기초,1179153,[168436],어떤 냄새,"[GN0500, GN0600]",[백현진],556642,백현진
572676,[GN0801],20191129,가볍고 수많은,10358776,[168436],반 줄,[GN0800],[백현진],572676,백현진
576100,"[GN1502, GN1501]",20050408,주먹이 운다 OST,301617,[168436],행복의 나라로,[GN1500],[백현진],576100,백현진
637548,"[GN0508, GN0501, GN0601, GN0503, GN0605]",20080519,Time Of Reflection,381052,[168436],눈물 닦은 눈물,"[GN0500, GN0600]",[백현진],637548,백현진


### 해당 제목 찾아오기

In [4]:
from itertools import chain

def find_title(titles):
    titles = titles.split()
    idx = []
    for title in titles:
        A = [x for x in train['plylst_title'] if title in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(train[train['plylst_title']  == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return train.iloc[idx_list,:]

In [5]:
find_title('마리')

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
107219,"[걸크러쉬, 내한, 앤마리, 셋리스트, 여성꿀보이스]",93518,Anne-Marie 앤마리 첫 내한공연 셋리스트,"[410502, 651374, 487877, 312257, 24553, 381874...",7,2020-04-23 22:04:00.000,걸크러쉬 내한 앤마리 셋리스트 여성꿀보이스,Anne-Marie 앤마리 첫 내한공연 셋리스트
29178,"[힙합, 힙합엘이, 블랙뮤직, HIPHOPLE, 흑인음악]",124222,"조금은 특별한 B급 감성, 리짓군즈와 오사마리","[141853, 148325, 2139, 427167, 590661, 199420,...",0,2019-12-19 14:41:31.000,힙합 힙합엘이 블랙뮤직 HIPHOPLE 흑인음악,"조금은 특별한 B급 감성, 리짓군즈와 오사마리"
69207,"[힙합, 힙합엘이, 알앤비]",28662,"조금은 특별한 B급 감성, 리짓군즈와 오사마리","[2139, 427167, 590661, 199420, 462515, 647821,...",8,2016-10-06 12:29:08.000,힙합 힙합엘이 알앤비,"조금은 특별한 B급 감성, 리짓군즈와 오사마리"
79515,"[감성, 해외, 잔잔한, 낭만, OST, 서정성, 힐링, 작곡가, 아련한, 스코어]",90958,"스크린에 채색한 클래식, 다리오 마리아넬리","[345357, 224081, 516495, 175689, 82346, 518627...",258,2019-03-11 10:34:21.000,감성 해외 잔잔한 낭만 OST 서정성 힐링 작곡가 아련한 스코어,"스크린에 채색한 클래식, 다리오 마리아넬리"
39927,"[휴식, 밤, 잔잔한, 새벽, 재즈, JAYJE, 힐링, 잠안올때, 불면증, Jazz]",55255,불면증에 양 한마리 재즈 한 곡,"[143660, 38650, 335431, 583384, 664013, 411147...",1037,2020-01-19 01:46:27.000,휴식 밤 잔잔한 새벽 재즈 JAYJE 힐링 잠안올때 불면증 Jazz,불면증에 양 한마리 재즈 한 곡
21451,"[앤마리, 내한공연, 공연, 세트리스트, 셋리스트, 고막여친]",144181,앤 마리 (Anne-Marie) 내한공연 예상 셋리스트,"[568354, 589965, 410502, 430840, 651374, 31225...",29,2019-03-21 14:37:21.000,앤마리 내한공연 공연 세트리스트 셋리스트 고막여친,앤 마리 (Anne-Marie) 내한공연 예상 셋리스트
32999,"[보석, 인디음악, 마리슈, 공감, 우리의_이야기, 너의_이야기, 내_이야기, 사랑]",132500,공감 마리슈,"[550986, 267439, 87373, 398832, 349860, 436686...",3,2018-11-13 09:01:29.000,보석 인디음악 마리슈 공감 우리의_이야기 너의_이야기 내_이야기 사랑,공감 마리슈
53491,"[오케스트라, 지휘자, 영국, 네빌마리너, 바로크]",702,한결 같은 연주를 들려준 지휘자 네빌 마리너,"[245224, 464285, 535310, 84996, 579101, 251790...",2,2020-02-25 15:33:22.000,오케스트라 지휘자 영국 네빌마리너 바로크,한결 같은 연주를 들려준 지휘자 네빌 마리너
72761,"[지역별, 분위기있는, 라틴, 월드뮤직, 추억, etc, 근사한, World, 감성...",139059,"멕시코 민중을 노래한 대중 음악, 2000년대 란체라/마리아치","[170644, 567992, 453316, 632621, 136013, 28317...",139,2018-08-22 13:58:40.000,지역별 분위기있는 라틴 월드뮤직 추억 etc 근사한 World 감성적인 중후한,"멕시코 민중을 노래한 대중 음악, 2000년대 란체라/마리아치"
84613,"[여름, 마리슈, 이정아, 여행, 제주도]",28527,이정아 마리슈 제주여행,"[379995, 511829, 140635, 380282, 306201, 51493...",1,2017-09-07 15:48:51.000,여름 마리슈 이정아 여행 제주도,이정아 마리슈 제주여행


### 해당 태그 찾아오기

In [6]:
from itertools import chain

def find_tag(tags):
    tags = tags.split()
    idx = []
    for tag in tags:
        A = [x for x in train['tags__'] if tag in x]
        A = list(set(A))

        
    
        for i in A:
            idx.append(train[train['tags__']  == i].index.tolist())
    
        idx_list = list(chain.from_iterable(idx))
    
    return train.iloc[idx_list,:]

In [7]:
find_tag('마리')

,tags,id,plylst_title,songs,like_cnt,updt_date,tags__,plylst_title__
107219,"[걸크러쉬, 내한, 앤마리, 셋리스트, 여성꿀보이스]",93518,Anne-Marie 앤마리 첫 내한공연 셋리스트,"[410502, 651374, 487877, 312257, 24553, 381874...",7,2020-04-23 22:04:00.000,걸크러쉬 내한 앤마리 셋리스트 여성꿀보이스,Anne-Marie 앤마리 첫 내한공연 셋리스트
32836,"[BEST30, 마리아칼라스, 오페라, 디아나담라우, 아리아, 안젤라게오르규, 호세...",63893,당신의 품격을 높일 명 오페라 아리아 BEST 30,"[490885, 281963, 696913, 555929, 689227, 70488...",354,2018-04-23 09:55:43.000,BEST30 마리아칼라스 오페라 디아나담라우 아리아 안젤라게오르규 호세카레라스 조수...,당신의 품격을 높일 명 오페라 아리아 BEST 30
21451,"[앤마리, 내한공연, 공연, 세트리스트, 셋리스트, 고막여친]",144181,앤 마리 (Anne-Marie) 내한공연 예상 셋리스트,"[568354, 589965, 410502, 430840, 651374, 31225...",29,2019-03-21 14:37:21.000,앤마리 내한공연 공연 세트리스트 셋리스트 고막여친,앤 마리 (Anne-Marie) 내한공연 예상 셋리스트
3656,"[러시아, 클래식, 웅장함, 마리스얀손스, 지휘자, 교향곡]",97243,탄탄한 앙상블의 마에스트로 마리스 얀손스,"[576634, 567521, 330814, 605638, 610491, 59317...",2,2020-02-25 16:56:55.000,러시아 클래식 웅장함 마리스얀손스 지휘자 교향곡,탄탄한 앙상블의 마에스트로 마리스 얀손스
58862,"[conductor, 오케스트라, 카를로마리아, classic, 레퍼토리, 필하모닉...",100631,"지휘계의 영원한 자유인, 카를로 마리아 줄리니","[158742, 227891, 563036, 630631, 596445, 42675...",19,2017-07-31 12:10:26.000,conductor 오케스트라 카를로마리아 classic 레퍼토리 필하모닉 심포니 웅...,"지휘계의 영원한 자유인, 카를로 마리아 줄리니"
53491,"[오케스트라, 지휘자, 영국, 네빌마리너, 바로크]",702,한결 같은 연주를 들려준 지휘자 네빌 마리너,"[245224, 464285, 535310, 84996, 579101, 251790...",2,2020-02-25 15:33:22.000,오케스트라 지휘자 영국 네빌마리너 바로크,한결 같은 연주를 들려준 지휘자 네빌 마리너
96454,"[힘내요, 태연, 마마무, 방탄소년단, 아이유, 성규, 앤마리, 레드벨벳, 엑소]",95697,힘이 들때 들으면 위로가 되어주는 노래,"[324208, 392638, 45351, 671533, 408035, 400079...",4,2020-04-12 15:16:01.000,힘내요 태연 마마무 방탄소년단 아이유 성규 앤마리 레드벨벳 엑소,힘이 들때 들으면 위로가 되어주는 노래
44056,"[동안, 소음밴드, 에디전, 위로가_되는_음악, 이설아, 리메인즈, 마리슈, 이정아...",70724,"숨겨진 좋은음악 추천하기#1[사랑/설렘,기분전환]","[468895, 151877, 622172, 149274, 28896, 592699...",6,2017-09-07 15:48:51.000,동안 소음밴드 에디전 위로가_되는_음악 이설아 리메인즈 마리슈 이정아 데일리노트,"숨겨진 좋은음악 추천하기#1[사랑/설렘,기분전환]"
34715,"[뉴릴리즈, 앤마리, 저스틴비버, 제임스블런트, 금주의신곡, 신곡, 팝송, 그린데이...",48765,[NPS] 2월 둘째주 출시 팝송 (매주 업데이트),"[589813, 81808, 556335, 241759, 630851, 660969...",56,2020-02-08 11:20:37.000,뉴릴리즈 앤마리 저스틴비버 제임스블런트 금주의신곡 신곡 팝송 그린데이 사운드소울,[NPS] 2월 둘째주 출시 팝송 (매주 업데이트)
23361,"[뷰티핸섬, 동안, 모브닝, 슈가레코드, 에디전, 입술을깨물다, 마리슈, 치즈스테레...",71005,Sugar Records Sampler #1,"[106752, 165613, 210394, 408293, 565552, 46575...",11,2017-09-07 15:48:51.000,뷰티핸섬 동안 모브닝 슈가레코드 에디전 입술을깨물다 마리슈 치즈스테레오 이정아 데일리노트,Sugar Records Sampler #1


---
---
---

### Mecab 사용자 사전 추가

- 가수 이름 추가 
  - 그냥 artist_name으로 나온다 다 정리 할거임
  - 중복 제거할거임
  - 로꼬 / 로꼬 유성은 / 로꼬 화사 이런식으로 저장될거임
  - 이렇게하면 12만개 정도의 이름이 나온다
  
- 숫자 + 단위명사 (SN+NNBC)
    - **회, 년, 월, 일** 이 경우는 twitter에서 Number로 인식해서 문제가 되지않는다
    - 문제가 되는 경우는 **살, 대**같은 단위명사는 Number로 인식을 하지 못한다는 것이다.

    - 그래서 **10살, 10대**같이 Number로 인식을 하지 못하는 단어들은 twitter에서 임의로 단어 사전으로 추가를 해줘야한다

    - 그렇게 하면 10살 10대 10회 10년 10월 10일 같이 나올 것이다 이 단어들의 품사는 모두 Number

    - 이렇게 나온 결과물들을 수사(NR)로 묶어주고 싶은데... 수사는 위의 단어는 수사에 해당시키지 못 할듯
        - 단위명사(NNBC)도 의존명사라서 가능은 할 거 같은데 잘 모르겠음

    - 그냥 **SN+NNBC라는 새로운 품사 파일을 생성해서 추가**함

In [4]:
# 사용자 사전 불러오기

# with open(r"C:\mecab\mecab-ko-dic\SN+NNBC.csv", 'r', encoding='utf-8') as f: 
#     file_data = sorted(f.readlines())
# file_data

In [9]:
age = [str(x)+'살' for x in list(range(0,130))]
generation = [str(x*10)+'대' for x in list(range(0,11))]
th = [str(x)+'회' for x in list(range(0,2000))]
year = [str(x)+'년' for x in list(range(1950,2051))]
month = [str(x)+'월' for x in list(range(1,13))]
day = [str(x)+'일' for x in list(range(1,32))]
hour = [str(x)+'시' for x in list(range(0,25))]
minute = [str(x)+'분' for x in list(range(0,61))]
sec = [str(x)+'초' for x in list(range(0,61))]

pp = np.concatenate([age, generation, th, year, month,day,hour,minute,sec])
pp

array(['0살', '1살', '2살', ..., '58초', '59초', '60초'], dtype='<U5')

In [7]:
# 사용자 사전 추가

# for X in pp:
#     file_data.append(f'{X},,,,SN+NNBC,*,T,{X},*,*,*,*,*\n')
#     with open(r"C:\mecab\mecab-ko-dic\SN+NNBC.csv", 'w', encoding='utf-8') as f: 
#         for line in file_data: 
#             f.write(line)

---
---
---

### PyKoSpacing / Py-hanspell

#### pykospacing 다운로드 및 연습

- 내가 볼 때는 이거보다 py-hamspell 성능이 더 좋은 것 같음

In [23]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\cv002\appdata\local\temp\pip-req-build-jybz1_uh
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2255682 sha256=b02008d4a210733bf6fea73f3ab03ac5b298958ebd5e4bbb13dbab3757b3b57e
  Stored in directory: C:\Users\cv002\AppData\Local\Temp\pip-ephem-wheel-cache-eerg6paa\wheels\79\a0\33\16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [13]:
from pykospacing import Spacing
spacing = Spacing()

In [26]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

spacing = Spacing()

kospacing_sent = spacing(new_sent) 

print(sent)
print(kospacing_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


#### py-hanspell 다운로드 및 연습

In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

In [ ]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)
print(kospacing_sent)

### Twitter

Twitter 사용자 사전 추가하고 사용자 사전 추가하기

추가된게 저장되지 않고 다른 노트북을 열때도 계속 추가를 해줘야 하는듯

In [ ]:
# Twitter 사용자 사전 추가 다운로드
# $ git clone https://github.com/lovit/customized_konlpy.git

# $ pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter
tagger = Twitter()

tagger.add_dictionary(['파이콘', '엔엘피', '아이오아이', '너무너무너무'], 'Noun')
tagger.add_dictionary(['8살'], 'Noun',force=True)

tagger.pos('8월')

---
---
---
### 자연어 처리 과정 / 클래스 비교 

In [24]:
text = '기리보이의노래는길이보인다 / 외않되 / 그래욬ㅋㅋㅋ'

# step 1. 띄어쓰기 및 맞춤법 체크
print('Py-hanspell : ',spell_checker.check(text).checked, '\n') # 띄어쓰기 + 맞춤법 해결 가능 
print('PyKoSpacing : ',spacing(text), '\n') # 띄어쓰기만 해결 가능 하지만 원하는 느낌이 아님
# 사용자 사전을 추가하는 방법을 모르겠음

# step 2. 정규화
# 정규화는 Okt랑 Twitter만 가능함 
# 그렇기 때문에 Kkma, Komoran, Hannanum Mecab은 이 과정에서는 사용 불가능 
print('Okt : ',okt.morphs(text, stem = True, norm = True), '\n') # 속도 느림 

print('Twitter : ',twitter.morphs(text, stem = True, norm = True), '\n') # Okt보다 속도 빠름 / 일회성으로 원하는 단어 추가 가능 

# step 3. 세부품사 변경
print('Mecab : ',m.pos(text), '\n') # 속도 가장 빠름

Py-hanspell :  기리보이의 노래는 길이 보인다 / 왜 안돼 / 그래욬ㅋㅋㅋ 

PyKoSpacing :  기리 보이의 노래는 길이 보인 다 / 외 않되 / 그래욬ㅋㅋㅋ 

Okt :  ['기리보이', '의', '노랗다', '이보', '인', '다', '/', '외', '않다', '/', '그렇다', 'ㅋㅋㅋ'] 

Twitter :  ['기리보이', '의', '노래', '는', '길', '이보', '인', '다', '/', '외', '않다', '/', '그래', '요', 'ㅋㅋㅋ'] 

Mecab :  [('기리보이', 'NNP'), ('의', 'JKG'), ('노래', 'NNG'), ('는', 'JX'), ('길', 'NNG'), ('이', 'JKS'), ('보인다', 'VV+EF'), ('/', 'SC'), ('외', 'NNG'), ('않', 'VV'), ('되', 'EC'), ('/', 'SC'), ('그래', 'VA+EC'), ('욬ㅋㅋㅋ', 'UNKNOWN')] 



---
### 자연어처리 

1. 특수기호 제거
2. Py-hanspell로 띄어쓰기 및 맞춤법 처리(붙여쓰기는 불가능)
3. Twitter를 통해 정규화 및 어간 추출 형태소 분리
4. Mecab으로 원하는 품사만 추출
5. 불용어 처리

In [5]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

okt = Okt()

twitter = Twitter()
# 0살부터 99살까지 명사로 추가해줌
twitter.add_dictionary([str(x) + '살' for x in range(1,100)], 'Noun')
twitter.add_dictionary(['10대','20대','30대','40대','50대','60대','70대'], 'Noun')
# 특정 가수 입력
twitter.add_dictionary('아이오아이', 'Noun')

m = Mecab(r'C:/mecab/mecab-ko-dic')

In [25]:
def konlpy_preprocessing(text, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트    
    
    
    text = re.sub('[^A-Za-z가-힣0-9]'," ",text)
    txt = spell_checker.check(text).checked
    
    
    # Twitter, Okt를 활용해서 
    # stem = True > 어간 추출
    # norm = True > 정규화 진행
    
#     morph = okt.morphs(txt, stem = True, norm = True)
    morph = twitter.morphs(txt, stem = True, norm = True)
        
    parts_of_speech = []
    # mecab으로 원하는 품사만 뽑아오기
    for mor in morph:
        for word, tag in m.pos(mor):
            
            # VV 포함해서 다시 돌리기
#             if tag in ['NNG', 'NNP', 'NNBC', 'VV','VA', 'XR', 'SN','MAG','SL']:
#                 parts_of_speech.append(word.lower())  
            
            if tag in ['NNG', 'NNP', 'VV','VA','XR', 'MAG', 'SL']:
                parts_of_speech.append(word)
            elif tag in 'SN+NNBC':
                parts_of_speech.append(word)    
                
#             if tag == "SN":
#                 if len(word) <= 4:
#                     global A
#                     A = str(word)
#                 elif len(word) >= 5:
#                     parts_of_speech.append(word)
#             elif tag == 'NNBC':
#                 global B
#                 B = word
#                 parts_of_speech.append(A + B)

#             else:
#                 parts_of_speech.append(word)
                
    
#     stopwords에 있는 단어 제거
    if removes_stopwords == True:
        parts_of_speech = [token for token in parts_of_speech if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        parts_of_speech = [w for w in parts_of_speech if not w in stops] 
    
    return parts_of_speech

In [26]:
konlpy_preprocessing('신나는', removes_stopwords = True, stop_words = stopword)

['신나']

---
---
---
####  태그 자연어 처리

In [27]:
text = train['tags__'].tolist()
text

['락',
 '추억 회상',
 '까페 잔잔한',
 '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
 '댄스',
 '운동 드라이브 Pop 트로피컬하우스 힐링 기분전환 2017 팝 트렌드 일렉',
 '짝사랑 취향저격 슬픔 고백 사랑 이별',
 '잔잔한 추억 회상',
 '일렉트로니카 포크 메탈 락 댄스 인디',
 '록 Metal 이일우 M에센셜 메탈 Rock 락',
 'kpop 댄스 걸그룹댄스 스트레스해소',
 '새해 여행 프로필음악 카카오톡 기분전환 소원 프로필 소망 다짐 카톡',
 '듣고 우울 힐링 이거 힘내',
 '힙합 느낌있는 밤 새벽 RnB 감각적인 드라이브 국내 그루브한',
 '가을 재즈',
 '락',
 '감성 질리지않는 나만알고싶은 Pop',
 '봄 설렘 사랑',
 '비오는날 누군가생각날때',
 '스밍 목록 폐막식 올림픽 엑소',
 '조용히 혼자 또는 새벽감성 고민 맥주한잔',
 '카페 재즈 잔잔한',
 'OST',
 '댄스',
 '감성 어쿠스틱 잔잔한 새벽 편안한 인디 밤',
 '잔잔한 밤 새벽',
 '힐링 드라이브 에너지 인디 여행',
 '듀엣 취향저격 달달 피쳐링 사랑 남녀',
 '까페 설렘 사랑',
 '힙합 신인 국힙 인디 신나는',
 '거슈윈 존윌리엄스 랩소디인블루 미국클래식 영화클래식 아메리칸클래식',
 '겨울 기분전환',
 '아침 빅밴드 상쾌한 재즈 밝은 스윙 스윙재즈',
 '휴식 힐링 설렘 사랑',
 '여름 일렉트로니카 매장음악 취향저격 댄스 드라이브 여행 기분전환 일렉 신나는',
 '까페 설렘 사랑',
 '잔잔한 설렘 사랑',
 '헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 '발라드',
 '겨울 크리스마스 캐롤',
 '위로 힐링',
 '기분전환 가을',
 '80년대 감성 90년대발라드 올드송 추억 휴식 90년대 옛날노래',
 '월드 연주 재즈',
 '잔잔한 밤 새벽',
 '슬픔 밤 이별 새벽',
 '클럽 스트레스',
 '휴식 주말 오후 나른한 Chillout 편안

In [28]:
tags = []
tag1 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[:20001]]
tags.append(tag1)
display(len(tag1))
display(len(tags))

20001

1

In [29]:
tag2 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[20001:40001]]
tags.append(tag2)
display(len(tag2))
display(len(tags))

20000

2

In [30]:
tag3 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[40001:60001]]
tags.append(tag3)
display(len(tag3))
display(len(tags))

20000

3

In [31]:
tag4 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[60001:80001]]
tags.append(tag4)
display(len(tag4))
display(len(tags))

20000

4

In [32]:
tag5 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[80001:100001]]
tags.append(tag5)
display(len(tag5))
display(len(tags))

20000

5

In [33]:
tag6 = [konlpy_preprocessing(txt, removes_stopwords=True, stop_words=stopword) for txt in text[100001:]]
tags.append(tag6)
display(len(tag6))
display(len(tags))

15070

6

In [34]:
# tag = list(chain.from_iterable(tags))
# tag

In [37]:
tag_df = pd.DataFrame(np.concatenate(tags))
tag_df.columns = ['tags__']
tag_df

,tags__
0,[락]
1,"[추억, 회상]"
2,"[카페, 잔잔]"
3,"[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ..."
4,[댄스]
...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]"
115067,[일렉]
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]"
115069,"[잔잔, 버스, 퇴근, 버스, Pop, 풍경, 퇴근길]"


In [38]:
tag_df.to_json('data/tag_konlpy.json', orient = 'records')

---
#### 타이틀 자연어 처리

In [44]:
test = train['plylst_title__'].tolist()
test

['여행같은 음악',
 '요즘 너 말야',
 '편하게, 잔잔하게 들을 수 있는 곡.-',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래 ㅋ',
 '2017 Pop Trend',
 '짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!',
 '멍청이.. 내맘도 몰라.',
 'DANCING IN THE MOON-LIGHT .01',
 '[록/메탈] Written by 이일우',
 '걸그룹 땐쓰쏭 ',
 '노래로 의지를 불태우자! "1일1다짐" st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합/알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면, 너도 내 생각 할까..',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때_새벽감성',
 'ᴡʜɪᴛᴇ : ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ',
 '추억의 명화를 만나는 시간 (영화 OST 모음)',
 '★☆기분좋은 여름날☆★',
 '지친 하루 끝, 힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때..(국외)',
 '기분좋은 햇살같은 숨겨진 명곡들 ❤',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지, 미국 대표 클래식',
 '겨울이 기다리며 미리 들어요^^~',
 '아침이 상쾌해지는 빅밴드의 스윙스윙 ♬',
 '여행가는 길 이런음악과 함께 고고씽!!!',
 '믿고듣는 하우스 일렉트로니카 POP #2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자!',
 '발라드로 감성을 발라드림(슬픔주의)',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '8~90년대 우리나라 라디오에서 흘러나오던 명곡들.',
 '영화, Jazz를 만나다 294',

In [46]:
title_to_tags = []
title_to_tag1 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[:20001]]
title_to_tags.append(title_to_tag1)
display(len(title_to_tag1))
display(len(title_to_tags))

20001

1

In [47]:
title_to_tag2 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[20001:40001]]
title_to_tags.append(title_to_tag2)
display(len(title_to_tag2))
display(len(title_to_tags))

20000

2

In [48]:
title_to_tag3 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[40001:60001]]
title_to_tags.append(title_to_tag3)
display(len(title_to_tag3))
display(len(title_to_tags))

20000

3

In [50]:
title_to_tag4 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[60001:80001]]
title_to_tags.append(title_to_tag4)
display(len(title_to_tag4))
display(len(title_to_tags))

# 가끔 이런 에러가 나옴 Expecting value: line 1 column 1 (char 0)

20000

4

In [51]:
title_to_tag5 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[80001:100001]]
title_to_tags.append(title_to_tag5)
display(len(title_to_tag5))
display(len(title_to_tags))

20000

5

In [53]:
title_to_tag6 = [konlpy_preprocessing(txt,removes_stopwords=True, stop_words=stopword) for txt in test[100001:]]
title_to_tags.append(title_to_tag6)
display(len(title_to_tag6))
display(len(title_to_tags))

15070

6

In [54]:
# title_to_tag = list(chain.from_iterable(title_to_tags))
# title_to_tag

In [55]:
title_to_tag_df = pd.DataFrame(np.concatenate(title_to_tags),columns=['plylist_title__'])
title_to_tag_df

,plylist_title__
0,"[여행, 음악]"
1,[요즘]
2,"[편하, 잔잔, 곡]"
3,"[크리스마스, 분위기, 흠뻑, 취하]"
4,"[추억, 노래]"
...,...
115066,"[METAL, E, SM, 2]"
115067,"[빠르, 리스, 위, 따끈따끈, 최신, 인기, EDM, 모음]"
115068,"[1, 눈물, 가리, 이야기]"
115069,"[퇴근, 버스, 편히, 하루, 마무리, POP]"


In [56]:
title_to_tag_df.to_json('data/title_to_tag_konlpy.json', orient = 'records')